In [1]:
# Wrap the old code into kooplearn
import schnetpack
import ml_confs
import torch

class SchNet(schnetpack.model.AtomisticModel):
    def __init__(self, configs: ml_confs.Configs):
        super().__init__(
            input_dtype_str="float32",
            postprocessors=None,
            do_postprocessing=False,
        )
        self.cutoff = configs.data.cutoff
        self.pwise_dist = schnetpack.atomistic.PairwiseDistances()
        self.radial_basis = schnetpack.nn.GaussianRBF(n_rbf=configs.network.n_rbf, cutoff=self.cutoff)
        self.net = schnetpack.representation.SchNet(
            n_atom_basis=configs.network.n_atom_basis, 
            n_interactions=configs.network.n_interactions,
            radial_basis=self.radial_basis,
            cutoff_fn=schnetpack.nn.CosineCutoff(self.cutoff)
        )
        self.final_lin = torch.nn.Linear(configs.network.n_atom_basis, configs.network.n_final_features)
        self.batch_norm = torch.nn.BatchNorm1d(configs.network.n_final_features, affine=False)
    def forward(self, inputs: dict[str, torch.Tensor]) -> dict[str, torch.Tensor]:
        inputs = self.pwise_dist(inputs)
        inputs = self.net(inputs)
        inputs['scalar_representation'] = self.final_lin(inputs['scalar_representation'])
        # n_atoms = inputs[schnetpack.properties.n_atoms][0] #Assuming constant number of atoms across the batch
        # n_final_features = inputs['scalar_representation'].shape[-1]
        # inputs['scalar_representation'] = self.batch_norm(
        #     inputs['scalar_representation'].view(-1, n_atoms, n_final_features).permute(0, 2, 1)
        # ).permute(0, 2, 1).reshape(-1, n_final_features)
        # inputs['scalar_representation'] =  inputs['scalar_representation']*torch.rsqrt(torch.tensor(n_final_features, dtype=torch.float32))
        return inputs


NameError: name 'JSONNamespace' is not defined